In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path

BASE_DIR = Path("/content/drive/Shareddrives/TissueMotionForecasting")

DATA_ROOT = BASE_DIR / "scared_data" / "test" / "test_dataset_8"

TOOLKIT_ROOT = BASE_DIR / "scared_toolkit"

print("BASE_DIR:", BASE_DIR)
print("DATA_ROOT:", DATA_ROOT)
print("TOOLKIT_ROOT:", TOOLKIT_ROOT)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BASE_DIR: /content/drive/Shareddrives/TissueMotionForecasting
DATA_ROOT: /content/drive/Shareddrives/TissueMotionForecasting/scared_data/test/test_dataset_8
TOOLKIT_ROOT: /content/drive/Shareddrives/TissueMotionForecasting/scared_toolkit


In [ ]:
import subprocess, os, sys

if not TOOLKIT_ROOT.exists():
    !git clone https://github.com/dimitrisPs/scared_toolkit.git "$TOOLKIT_ROOT"
else:
    print("scared_toolkit already exists at", TOOLKIT_ROOT)


scared_toolkit already exists at /content/drive/Shareddrives/TissueMotionForecasting/scared_toolkit


In [ ]:
%cd "$TOOLKIT_ROOT"
!pip install -q numpy tifffile plyfile opencv-python tqdm


In [ ]:
from pathlib import Path

keyframes = sorted(DATA_ROOT.glob("keyframe_*"))
print("Found keyframes in dataset_1:")
for kf in keyframes:
    print("  ", kf)

print("Total:", len(keyframes))


Found keyframes in dataset_1:
   /content/drive/Shareddrives/TissueMotionForecasting/scared_data/test/test_dataset_8/keyframe_0
   /content/drive/Shareddrives/TissueMotionForecasting/scared_data/test/test_dataset_8/keyframe_1
   /content/drive/Shareddrives/TissueMotionForecasting/scared_data/test/test_dataset_8/keyframe_2
   /content/drive/Shareddrives/TissueMotionForecasting/scared_data/test/test_dataset_8/keyframe_3
   /content/drive/Shareddrives/TissueMotionForecasting/scared_data/test/test_dataset_8/keyframe_4
Total: 5


In [ ]:
%cd "$TOOLKIT_ROOT"

from tqdm.auto import tqdm

for kf in tqdm(keyframes, desc="Processing keyframes"):
    print("\n=== Processing", kf, "===")
    # For each keyframe:
    #  - reads endoscope_calibration.yaml inside that folder
    #  - uses data/rgb.mp4 + scene_points.tar.gz + frame_data.tar.gz
    #  - generates rectified views + disparity maps under keyframe/data/
    !python -m scripts.extract_sequence_dataset \
        "$kf" \
        --depth \
        --disparity


In [ ]:
import numpy as np
import imageio.v2 as imageio
import matplotlib.cm as cm
from pathlib import Path

DATA_ROOT = Path("/content/drive/Shareddrives/TissueMotionForecasting") / "scared_data" / "train" / "dataset_3" / "dataset_3"
disparity_pngs = sorted(DATA_ROOT.glob("keyframe_2/data/disparity/*.png"))
print("Found disparity pngs:", len(disparity_pngs))

SCALE_FACTOR = 256.0

def save_raft_style_disparity(png_path: Path, flip_for_red_near=True):
    disp_u16 = imageio.imread(png_path)
    disp = disp_u16.astype(np.float32) / SCALE_FACTOR

    disp[disp_u16 == 0] = np.nan

    valid = np.isfinite(disp)
    if valid.any():
        vmin, vmax = np.nanpercentile(disp[valid], [5, 95])
    else:
        vmin, vmax = 0.0, 1.0

    norm = np.clip((disp - vmin) / (vmax - vmin + 1e-6), 0, 1)
    norm[np.isnan(norm)] = 0.0

    cmap = cm.get_cmap("turbo")
    color = (cmap(norm)[..., :3] * 255).astype(np.uint8)

    out_path = png_path.with_name("colored_" + png_path.stem + ".png")
    imageio.imwrite(out_path, color)
    return out_path

raft_color_paths = [save_raft_style_disparity(p) for p in disparity_pngs]


In [ ]:
depth_pngs = sorted(DATA_ROOT.glob("keyframe_*/data/depthmap/*.png"))
print("Found depth pngs:", len(depth_pngs))

def save_flipped_depth(png_path: Path):
    depth_uint16 = imageio.imread(png_path)
    depth = depth_uint16.astype(np.float32) / SCALE_FACTOR

    depth[depth_uint16 == 0] = np.nan

    valid = np.isfinite(depth)
    if valid.any():
        vmin, vmax = np.nanpercentile(depth[valid], [5, 95])
    else:
        vmin, vmax = 0.0, 1.0

    norm = np.clip((depth - vmin) / (vmax - vmin + 1e-6), 0, 1)
    norm = 1.0 - norm
    norm[np.isnan(norm)] = 0.0

    depth_gray = (norm * 255).astype(np.uint8)

    out_path = png_path.with_name("corrected" + png_path.stem + ".png")
    imageio.imwrite(out_path, depth_gray)
    return out_path

depth_flipped_paths = [save_flipped_depth(p) for p in depth_pngs]


In [ ]:
import numpy as np
import imageio.v2 as imageio
from pathlib import Path
from tqdm.auto import tqdm


BASE = Path("/content/drive/Shareddrives/TissueMotionForecasting")
DATA_ROOT = BASE / "scared_data" / "train" / "dataset_2"

KEYFRAME_ID = "keyframe_4"

KF_DIR = DATA_ROOT / KEYFRAME_ID
LEFT_DIR   = KF_DIR / "data" / "left_rectified"
RIGHT_DIR  = KF_DIR / "data" / "right_rectified"
DISP_DIR   = KF_DIR / "data" / "disparity"
DEPTH_DIR  = KF_DIR / "data" / "depthmap"

OUTPUT_VIDEO = KF_DIR / f"{KEYFRAME_ID}_2x2_grid_colored.mp4"
FPS = 10

print("Keyframe dir:", KF_DIR)
print("Output video:", OUTPUT_VIDEO)


left_files  = sorted(LEFT_DIR.glob("*.png")) + sorted(LEFT_DIR.glob("*.tif*"))
right_files = sorted(RIGHT_DIR.glob("*.png")) + sorted(RIGHT_DIR.glob("*.tif*"))

disp_color_files = sorted(DISP_DIR.glob("colored_*.png"))

depth_color_files = sorted(DEPTH_DIR.glob("corrected*.png"))

print("left_rectified frames :", len(left_files))
print("right_rectified frames:", len(right_files))
print("colored disparity     :", len(disp_color_files))
print("corrected depth       :", len(depth_color_files))

n_frames = min(len(left_files), len(right_files), len(disp_color_files), len(depth_color_files))
left_files       = left_files[:n_frames]
right_files      = right_files[:n_frames]
disp_color_files = disp_color_files[:n_frames]
depth_color_files= depth_color_files[:n_frames]
print("Using n_frames =", n_frames)

if n_frames == 0:
    raise RuntimeError("No frames to write – check that colored_*.png and corrected*.png exist.")

def read_rgb_any(path: Path):
    """Read image and ensure H×W×3 uint8."""
    img = imageio.imread(path)
    img = np.asarray(img)
    if img.ndim == 2:
        img = np.stack([img]*3, axis=-1)
    if img.dtype == np.uint16:
        if img.max() > 0:
            img = (img / img.max() * 255.0).astype(np.uint8)
        else:
            img = img.astype(np.uint8)
    elif img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)
    return img

def make_2x2_grid(left_rgb, right_rgb, disp_rgb, depth_rgb):
    """
    2x2 layout:
        [ left_rgb   | right_rgb ]
        [ disp_rgb   | depth_rgb ]
    """
    left_rgb  = read_rgb_any(left_rgb)  if isinstance(left_rgb, Path)  else left_rgb
    right_rgb = read_rgb_any(right_rgb) if isinstance(right_rgb, Path) else right_rgb
    disp_rgb  = read_rgb_any(disp_rgb)  if isinstance(disp_rgb, Path)  else disp_rgb
    depth_rgb = read_rgb_any(depth_rgb) if isinstance(depth_rgb, Path) else depth_rgb

    H, W, _ = left_rgb.shape

    def ensure_hw3(x):
        x = np.asarray(x)
        if x.ndim == 2:
            x = np.stack([x]*3, axis=-1)
        return x.reshape(H, W, 3)

    right_rgb = ensure_hw3(right_rgb)
    disp_rgb  = ensure_hw3(disp_rgb)
    depth_rgb = ensure_hw3(depth_rgb)

    top_row    = np.concatenate([left_rgb, right_rgb], axis=1)   # (H, 2W, 3)
    bottom_row = np.concatenate([disp_rgb, depth_rgb], axis=1)   # (H, 2W, 3)
    grid       = np.concatenate([top_row, bottom_row], axis=0)   # (2H, 2W, 3)
    return grid.astype(np.uint8)

print("Writing video to:", OUTPUT_VIDEO)
writer = imageio.get_writer(OUTPUT_VIDEO, fps=FPS, codec="libx264")

for lf, rf, df, zf in tqdm(
    zip(left_files, right_files, disp_color_files, depth_color_files),
    total=n_frames,
    desc="Frames"
):
    frame = make_2x2_grid(lf, rf, df, zf)
    writer.append_data(frame)

writer.close()
print("Done:", OUTPUT_VIDEO)


Keyframe dir: /content/drive/Shareddrives/TissueMotionForecasting/scared_data/train/dataset_2/keyframe_4
Output video: /content/drive/Shareddrives/TissueMotionForecasting/scared_data/train/dataset_2/keyframe_4/keyframe_4_2x2_grid_colored.mp4
left_rectified frames : 2114
right_rectified frames: 2114
colored disparity     : 2114
corrected depth       : 2114
Using n_frames = 2114
Writing video to: /content/drive/Shareddrives/TissueMotionForecasting/scared_data/train/dataset_2/keyframe_4/keyframe_4_2x2_grid_colored.mp4


Frames:   0%|          | 0/2114 [00:00<?, ?it/s]

Done: /content/drive/Shareddrives/TissueMotionForecasting/scared_data/train/dataset_2/keyframe_4/keyframe_4_2x2_grid_colored.mp4
